In [198]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from textblob import TextBlob

In [199]:
train_data = pd.read_csv('train_data.csv')

C:\Users\fatemeh\AppData\Local\Temp\ipykernel_14900\379951711.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv')


In [200]:
def clean_text(text):
    
    #Remove links
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text =re.sub(url_pattern, '', text)

    #Removing Numbers
    text = re.sub(r'\d+', '', text)

    #Removing Punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Removing Whitespace
    text = ' '.join(text.split())

    #Removing Special Characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    #Remove html tags
    clean = re.compile('<.*?>')
    text= re.sub(clean, '', text)


    return text

In [201]:
train_data["cleaned_text"]= train_data['reviewText'].apply(clean_text)

In [202]:
def lowercase_text(text):
    text = text.lower()
    return text

In [203]:
train_data["cleaned_text"]= train_data['cleaned_text'].apply(lowercase_text)

In [204]:
def tokenize_text(text):
    
    text = word_tokenize(text)
    return text

In [205]:
train_data["cleaned_text"]= train_data['cleaned_text'].apply(tokenize_text)

In [206]:
nltk.download('punkt')
nltk.download('stopwords')

def remove_stopwords(tokens):

    stop_words = set(stopwords.words('english'))
    text =[w for w in tokens if not w.lower() in stop_words]
    return " ".join(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fatemeh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatemeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [207]:
train_data["cleaned_text"]= train_data['cleaned_text'].apply(remove_stopwords)

In [208]:
nlp = spacy.load("en_core_web_sm")

def lemmatization(text):

    # Process the sentence with spaCy
    doc = nlp(text)

    # Extract the lemmas
    lemmatized_tokens = [token.lemma_ for token in doc]

    return lemmatized_tokens

In [209]:
train_data["cleaned_text"]= train_data['cleaned_text'].apply(lemmatization)

In [210]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')

target_words = ['guarantee', 'warranty']

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())
    print(synonyms)
    return synonyms

# Find synonyms for the target words
target_synonyms = set()
for word in target_words:
    target_synonyms.update(get_synonyms(word))


def check_for_similar_words(tokens):
    
    # Correct spelling using TextBlob
    corrected_tokens = [str(TextBlob(token).correct()) for token in tokens]

    # Check for similar words including misspellings
    for token in corrected_tokens:
        if token in target_words or token in target_synonyms:
            return True

{'warrantee', 'ensure', 'insure', 'guarantee', 'undertake', 'secure', 'vouch', 'assure', 'warrant', 'warranty', 'guaranty'}
{'warrant', 'warranty', 'guarantee', 'warrantee'}


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fatemeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fatemeh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [211]:
train_data["result"]= train_data['cleaned_text'].apply(check_for_similar_words)

In [212]:
# filtering reviews
rslt_df = train_data.loc[train_data['result'] == True]

# average_score
average_score = rslt_df.groupby('asin')['overall'].mean().reset_index()
print(average_score)

         asin  overall
0  0972683275      5.0
1  9806010728      4.0
2  B000001OM4      4.0
3  B000001OM5      5.0
4  B000001ON6      5.0
